# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Spark Data Sources (Files) and Transformations** </center>
---
**Profesor**: Pablo Camarillo Ramirez

# Find the PySpark Installation

In [1]:
import findspark
findspark.init()

# Create SparkSession

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://22410cdc64b0:7077") \
    .config("spark.jars", "/opt/spark/work-dir/jars/postgresql-42.7.8.jar") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

25/09/30 15:08:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Define the schema

In [3]:
from pcamarillor.spark_utils import SparkUtils
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema

StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

## Load CSV

In [4]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("/opt/spark/work-dir/data/airlines/directory/airlines_flights_data.csv")

df_airlines.show(n=5)

[Stage 0:>                                                          (0 + 1) / 1]

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

## Data Cleaning

In [5]:
from pyspark.sql.functions import trim, col, count, isnull, when
print(f"number of records before cleaning: {df_airlines.count()}")
# Get number of null values for each column before cleaning 
df_airlines.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

# Perform data cleaning with trim (column by column)
airlines_clean = df_airlines \
        .dropDuplicates(["index"]) \
        .withColumn("airline", trim("airline")) \
        .withColumn("source_city", trim("source_city")) \
        .withColumn("destination_city", trim("destination_city")) \
        .filter(col("price").isNotNull())

# Simply using dropna()
airlines_clean_v2 = df_airlines.dropna()

print(f"number of records after cleaning with trim: {airlines_clean.count()}")
airlines_clean.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

print(f"number of records after cleaning with dropna: {airlines_clean_v2.count()}")
airlines_clean_v2.select([count(when(isnull(c[0]) | col(c[0]).isNull(), c[0])).alias(c[0]) for c in airlines_schema_columns]).show()

number of records before cleaning: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



number of records after cleaning with trim: 300153


+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



number of records after cleaning with dropna: 300153


[Stage 22:=============================>                            (1 + 1) / 2]

+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|index|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|    0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+-----+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



## Create columns

In [6]:
from pyspark.sql.functions import when, lit

airlines_t1 = airlines_clean_v2.withColumn(
    "stops_numeric",
    when(airlines_clean_v2.stops == "zero", lit(0))
    .when(airlines_clean_v2.stops == "one", lit(1))
    .when(airlines_clean_v2.stops == "two_or_more", lit(2))
)

airlines_t1.show(n=1)


+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+-------------+
|index| airline| flight|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|price|stops_numeric|
+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+-------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|       Night|          Mumbai|Economy|    2.17|        1| 5953|            0|
+-----+--------+-------+-----------+--------------+-----+------------+----------------+-------+--------+---------+-----+-------------+
only showing top 1 row


## Create a new column called route

In [7]:
from pyspark.sql.functions import concat_ws

airlines_t2 = airlines_t1.withColumn(
    "route",
    concat_ws(" → ", "source_city", "destination_city")
)

airlines_t2.select("source_city", "destination_city", "route").show(5)


+-----------+----------------+--------------+
|source_city|destination_city|         route|
+-----------+----------------+--------------+
|      Delhi|          Mumbai|Delhi → Mumbai|
|      Delhi|          Mumbai|Delhi → Mumbai|
|      Delhi|          Mumbai|Delhi → Mumbai|
|      Delhi|          Mumbai|Delhi → Mumbai|
|      Delhi|          Mumbai|Delhi → Mumbai|
+-----------+----------------+--------------+
only showing top 5 rows


## Transform departure_time and arrival_time

In [8]:
from pyspark.sql.functions import when, col

airlines_t3 = airlines_t2.withColumn(
    "departure_code",
    when(col("departure_time") == "Early_Morning", 0)
    .when(col("departure_time") == "Morning", 1)
    .when(col("departure_time") == "Afternoon", 2)
    .when(col("departure_time") == "Evening", 3)
    .when(col("departure_time") == "Night", 4)
)

airlines_t3 = airlines_t3.withColumn(
    "arrival_code",
    when(col("arrival_time") == "Early_Morning", 0)
    .when(col("arrival_time") == "Morning", 1)
    .when(col("arrival_time") == "Afternoon", 2)
    .when(col("arrival_time") == "Evening", 3)
    .when(col("arrival_time") == "Night", 4)
)

airlines_t3.select("departure_time", "departure_code", "arrival_time", "arrival_code").show(5)


+--------------+--------------+-------------+------------+
|departure_time|departure_code| arrival_time|arrival_code|
+--------------+--------------+-------------+------------+
|       Evening|             3|        Night|           4|
| Early_Morning|             0|      Morning|           1|
| Early_Morning|             0|Early_Morning|           0|
|       Morning|             1|    Afternoon|           2|
|       Morning|             1|      Morning|           1|
+--------------+--------------+-------------+------------+
only showing top 5 rows


## Add a new column is_expensive

In [9]:
from pyspark.sql.functions import when, col

airlines_t4 = airlines_t3.withColumn(
    "is_expensive",
    when(col("price") > 6000, True).otherwise(False)
)

airlines_t4.select("price", "is_expensive").show(5)


+-----+------------+
|price|is_expensive|
+-----+------------+
| 5953|       false|
| 5953|       false|
| 5956|       false|
| 5955|       false|
| 5955|       false|
+-----+------------+
only showing top 5 rows


## PostgreSQL

In [10]:
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"
table_name = "airlines_transformed"

airlines_t4.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", table_name) \
    .option("user", "postgres") \
    .option("password", "Admin@1234") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("DataFrame successfully written into a PosgreSQL DB !")

[Stage 29:=============================>                            (1 + 1) / 2]

DataFrame successfully written into a PosgreSQL DB !


In [11]:
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"
db_properties = {
      "user": "postgres",      
      "password": "Admin@1234",
      "driver": "org.postgresql.Driver"
  }

df = spark.read \
    .jdbc(url=jdbc_url, table=table_name, properties=db_properties)

df.printSchema()
df.show(5, truncate=False)

root
 |-- index: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- source_city: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stops: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- class: string (nullable = true)
 |-- duration: float (nullable = true)
 |-- days_left: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- stops_numeric: integer (nullable = true)
 |-- route: string (nullable = true)
 |-- departure_code: integer (nullable = true)
 |-- arrival_code: integer (nullable = true)
 |-- is_expensive: boolean (nullable = true)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+-------------+--------------+--------------+------------+------------+
|index|airline |flight |source_city|departure_time|stops|arrival_time |destination_city|class

## Get the average price per airline

In [12]:
from pyspark.sql.functions import avg, min, max, count

airlines_t4.groupBy("airline") \
           .agg(avg("price").alias("avg_price")) \
           .show()


[Stage 31:=============================>                            (1 + 1) / 2]

+---------+------------------+
|  airline|         avg_price|
+---------+------------------+
|   Indigo| 5324.216303339517|
| SpiceJet| 6179.278881367218|
|Air_India| 23507.01911190229|
|  AirAsia|4091.0727419555224|
| GO_FIRST| 5652.007595045959|
|  Vistara| 30396.53630170735|
+---------+------------------+



## Average duration per route

In [13]:
airlines_t4.groupBy("route") \
           .agg(avg("duration").alias("avg_duration")) \
           .show()


[Stage 34:=============================>                            (1 + 1) / 2]

+--------------------+------------------+
|               route|      avg_duration|
+--------------------+------------------+
|Hyderabad → Banga...| 12.09331678643705|
|    Mumbai → Kolkata|12.836848115489666|
|    Mumbai → Chennai|12.665900287564627|
|  Mumbai → Hyderabad|13.263310412247066|
|  Mumbai → Bangalore|11.612022516178817|
|   Bangalore → Delhi|  9.77995566082195|
| Kolkata → Bangalore| 13.79294687524098|
|   Hyderabad → Delhi|10.829816602522587|
| Bangalore → Chennai|14.480207509137166|
|  Bangalore → Mumbai| 10.90507225639642|
|      Mumbai → Delhi|  9.81805726844943|
|  Hyderabad → Mumbai|11.962923295795918|
|   Kolkata → Chennai|14.774181563782903|
| Kolkata → Hyderabad|13.853107514948396|
|   Delhi → Bangalore| 10.35412503844018|
|      Delhi → Mumbai|10.367774213738123|
| Hyderabad → Chennai|13.293238468912078|
|Bangalore → Hyder...|14.162432783513621|
|     Kolkata → Delhi| 11.60498857561711|
|   Delhi → Hyderabad|12.518350118710492|
+--------------------+------------

## Minimum and maximum price per airline.

In [14]:
airlines_t4.groupBy("airline") \
           .agg(
               min("price").alias("min_price"),
               max("price").alias("max_price")
           ).show()


[Stage 37:=============================>                            (1 + 1) / 2]

+---------+---------+---------+
|  airline|min_price|max_price|
+---------+---------+---------+
|   Indigo|     1105|    31952|
| SpiceJet|     1106|    34158|
|Air_India|     1526|    90970|
|  AirAsia|     1105|    31917|
| GO_FIRST|     1105|    32803|
|  Vistara|     1714|   123071|
+---------+---------+---------+



## Count flights by departure_time category

In [15]:
airlines_t4.groupBy("departure_code") \
           .count() \
           .show()


[Stage 40:=============================>                            (1 + 1) / 2]

+--------------+-----+
|departure_code|count|
+--------------+-----+
|          NULL| 1306|
|             1|71146|
|             3|65102|
|             4|48015|
|             2|47794|
|             0|66790|
+--------------+-----+



In [16]:
sc.stop()